# Transformation to DataFrames

> Split-apply-combine

In [ ]:
using DataFrames

## Grouping a data frame

In [ ]:
x = DataFrame(id=[1,2,3,4,1,2,3,4], id2=[1,2,1,2,1,2,1,2], v=rand(8))

In [ ]:
groupby(x, :id)

In [ ]:
groupby(x, [])

In [ ]:
gx2 = groupby(x, [:id, :id2])

In [ ]:
parent(gx2) # get the parent DataFrame 

In [ ]:
vcat(gx2...) # back to the DataFrame, but in a different order of rows than the original

In [ ]:
DataFrame(gx2) # the same

In [ ]:
DataFrame(gx2, keepkeys=false) # drop grouping columns when creating a data frame

In [ ]:
groupcols(gx2) # vector of names of grouping variables

In [ ]:
valuecols(gx2) # and non-grouping variables

In [ ]:
groupindices(gx2) # group indices in parent(gx2)

In [ ]:
kgx2 = keys(gx2)

You can index into a `GroupedDataFrame` like to a vector or to a dictionary.
The second form acceps `GroupKey`, `NamedTuple` or a `Tuple`

In [ ]:
gx2

In [ ]:
k = keys(gx2)[1]

In [ ]:
ntk = NamedTuple(k)

In [ ]:
tk = Tuple(k)

the operations below produce the same result and are fast

In [ ]:
gx2[1]

In [ ]:
gx2[k]

In [ ]:
gx2[ntk]

In [ ]:
gx2[tk]

handling missing values

In [ ]:
x = DataFrame(id = [missing, 5, 1, 3, missing], x = 1:5)

In [ ]:
groupby(x, :id) # by default groups include mising values and their order is not guaranteed

In [ ]:
groupby(x, :id, sort=true, skipmissing=true) # but we can change it; now they are sorted

In [ ]:
groupby(x, :id, sort=false) # and now they are in the order they appear in the source data frame

## Performing transformations by group using `combine`, `select`, `select!`, `transform`, and `transform!`

In [ ]:
using Statistics
using Chain

In [ ]:
ENV["LINES"] = 15 # reduce the number of rows in the output

In [ ]:
x = DataFrame(id=rand('a':'d', 100), v=rand(100))

In [ ]:
# apply a function to each group of a data frame
# combine keeps as many rows as are returned from the function
@chain x begin
    groupby(:id)
    combine(:v => mean)
end

In [ ]:
x.id2 = axes(x, 1)

In [ ]:
# select and transform keep as many rows as are in the source data frame and in correct order
# additionally transform keeps all columns from the source
@chain x begin
    groupby(:id)
    transform(:v => mean)
end

In [ ]:
# note that combine reorders rows by group of GroupedDataFrame
@chain x begin
    groupby(:id)
    combine(:id2, :v => mean)
end

In [ ]:
# we give a custom name for the result column
@chain x begin
    groupby(:id)
    combine(:v => mean => :res)
end

In [ ]:
# you can have multiple operations
@chain x begin
    groupby(:id)
    combine(:v => mean => :res1, :v => sum => :res2, nrow => :n)
end

Additional notes:
* `select!` and `transform!` perform operations in-place
* The general syntax for transformation is `source_columns => function => target_column`
* if you pass multiple columns to a function they are treated as positional arguments
* `ByRow` and `AsTable` work exactly like discussed for operations on data frames in 05_columns.ipynb
* you can automatically groupby again the result of `combine`, `select` etc. by passing `ungroup=false` keyword argument to them
* similarly `keepkeys` keyword argument allows you to drop grouping columns from the resulting data frame

It is also allowed to pass a function to all these functions (also - as a special case, as a first argument). In this case the return value can be a table. In particular it allows for an easy dropping of groups if you return an empty table from the function.

If you pass a function you can use a `do` block syntax. In case of passing a function it gets a `SubDataFrame` as its argument.

Here is an example:

In [ ]:
combine(groupby(x, :id)) do sdf
    n = nrow(sdf)
    n < 25 ? DataFrame() : DataFrame(n=n) # drop groups with low number of rows
end

You can also produce multiple columns in a single operation, e.g.:

In [ ]:
df = DataFrame(id=[1,1,2,2], val=[1,2,3,4])

In [ ]:
@chain df begin
    groupby(:id)
    combine(:val => (x -> [x]) => AsTable)
end

In [ ]:
@chain df begin
    groupby(:id)
    combine(:val => (x -> [x]) => [:c1, :c2])
end

It is easy to unnest the column into multiple columns, e.g.

In [ ]:
df = DataFrame(a=[(p=1, q=2), (p=3, q=4)])

In [ ]:
select(df, :a => AsTable)

In [ ]:
df = DataFrame(a=[[1, 2], [3, 4]])

In [ ]:
select(df, :a => AsTable) # automatic column names generated

In [ ]:
select(df, :a => [:C1, :C2]) # custom column names generated

Finally, observe that one can conveniently apply multiple transformations using broadcasting:

In [ ]:
df = DataFrame(id=repeat(1:10, 10), x1=1:100, x2=101:200)

In [ ]:
@chain df begin
    groupby(:id)
    combine([:x1, :x2] .=> minimum)
end

In [ ]:
@chain df begin
    groupby(:id)
    combine([:x1, :x2] .=> [minimum maximum])
end

## Aggregation of a data frame using `mapcols`

In [ ]:
x = DataFrame(rand(10, 10), :auto)

In [ ]:
mapcols(mean, x)

## Mapping rows and columns using `eachcol` and `eachrow`

In [ ]:
map(mean, eachcol(x)) # map a function over each column and return a vector

In [ ]:
# an iteration returns a Pair with column name and values
foreach(c -> println(c[1], ": ", mean(c[2])), pairs(eachcol(x)))

In [ ]:
# now the returned value is DataFrameRow which works as a NamedTuple but is a view to a parent DataFrame
map(r -> r.x1/r.x2, eachrow(x))

In [ ]:
# it prints like a data frame, only the caption is different so that you know the type of the object
er = eachrow(x)

In [ ]:
er.x1 # you can access columns of a parent data frame directly

In [ ]:
# it prints like a data frame, only the caption is different so that you know the type of the object
ec = eachcol(x)

In [ ]:
ec.x1 # you can access columns of a parent data frame directly

## Transposing

you can transpose a data frame using `permutedims`:

In [ ]:
df = DataFrame(reshape(1:12, 3, 4), :auto)

In [ ]:
df.names = ["a", "b", "c"]

In [ ]:
permutedims(df, :names)